# Nazaré Marine Analytics – Notebook Analysis

This notebook uses **Snowpark for Python** to:

- Connect to Snowflake  
- Load the `STORM_MARINE_CLEAN` table  
- Explore the marine dataset (EDA)  
- Prepare a clean hourly time-series for forecasting

In [ ]:
import os
from snowflake.snowpark import Session, functions as F
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Use the same env vars as your scripts / .env
conn_params = {
    "account": os.getenv("SNOW_ACCOUNT"),
    "user": os.getenv("SNOW_USER"),
    "password": os.getenv("SNOW_PASSWORD"),
    "role": os.getenv("SNOW_ROLE", "SYSADMIN"),
    "warehouse": os.getenv("SNOW_WAREHOUSE", "COMPUTE_WH"),
    "database": os.getenv("SNOW_DATABASE", "MARINE_DB"),
    "schema": os.getenv("SNOW_SCHEMA", "NAZARE_SCHEMA"),
}

session = Session.builder.configs(conn_params).create()
session.sql("SELECT CURRENT_ROLE(), CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA()").show()


----------------------------------------------------------------------------------------
|"CURRENT_ROLE()"  |"CURRENT_WAREHOUSE()"  |"CURRENT_DATABASE()"  |"CURRENT_SCHEMA()"  |
----------------------------------------------------------------------------------------
|ACCOUNTADMIN      |SNOWFLAKE_LEARNING_WH  |MARINE_DB             |NAZARE_SCHEMA       |
----------------------------------------------------------------------------------------



In [2]:
# Load the clean table
df_snow = session.table("STORM_MARINE_CLEAN")

# Show schema
print("Schema:")
for col in df_snow.schema.fields:
    print(f"- {col.name} ({col.datatype})")

# Sample 10 rows
df_snow.limit(10).show()


Schema:
- TIMESTAMP (TimestampType(timezone=TimestampTimeZone('ltz')))
- WAVE_HEIGHT (DoubleType())
- SWELL_HEIGHT (DoubleType())
- WIND_SPEED (DoubleType())
- WATER_TEMPERATURE (DoubleType())
- LAT (DoubleType())
- LON (DoubleType())
- SOURCE (StringType(16777216))
- INGESTED_AT (TimestampType(timezone=TimestampTimeZone('ltz')))
- HOUR (LongType())
- DAYOFWEEK (LongType())
- ROLLING_WAVE_3H (DoubleType())
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TIMESTAMP"                |"WAVE_HEIGHT"  |"SWELL_HEIGHT"  |"WIND_SPEED"  |"WATER_TEMPERATURE"  |"LAT"     |"LON"      |"SOURCE"    |"INGESTED_AT"                     |"HOUR"  |"DAYOFWEEK"  |"ROLLING_WAVE_3H"   |
----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
summary_df = session.sql("""
    SELECT
        COUNT(*) AS row_count,
        MIN(TIMESTAMP) AS min_ts,
        MAX(TIMESTAMP) AS max_ts
    FROM STORM_MARINE_CLEAN
""").to_pandas()

summary_df


,ROW_COUNT,MIN_TS,MAX_TS
0,73,2025-12-04 03:00:00-08:00,2025-12-07 03:00:00-08:00


In [4]:
# Snowpark aggregation
stats_snow = (
    df_snow.select(
        F.mean("WAVE_HEIGHT").alias("avg_wave"),
        F.max("WAVE_HEIGHT").alias("max_wave"),
        F.mean("SWELL_HEIGHT").alias("avg_swell"),
        F.mean("WIND_SPEED").alias("avg_wind"),
        F.mean("WATER_TEMPERATURE").alias("avg_temp")
    )
)

stats_snow.to_pandas()


,AVG_WAVE,MAX_WAVE,AVG_SWELL,AVG_WIND,AVG_TEMP
0,3.911233,4.69,3.908493,3.662329,15.763014


In [5]:
# Pull a reasonable window (e.g., last 7 days) to avoid huge downloads
pdf = session.sql("""
    WITH MAX_TS AS (
      SELECT MAX(TIMESTAMP) AS MAX_T FROM STORM_MARINE_CLEAN
    )
    SELECT *
    FROM STORM_MARINE_CLEAN c, MAX_TS m
    WHERE c.TIMESTAMP >= DATEADD('day', -7, m.MAX_T)
    ORDER BY c.TIMESTAMP
""").to_pandas()

pdf["timestamp"] = pd.to_datetime(pdf["TIMESTAMP"])
pdf.head()


,TIMESTAMP,WAVE_HEIGHT,SWELL_HEIGHT,WIND_SPEED,WATER_TEMPERATURE,LAT,LON,SOURCE,INGESTED_AT,HOUR,DAYOFWEEK,ROLLING_WAVE_3H,MAX_T,timestamp
0,2025-12-04 03:00:00-08:00,3.69,3.46,5.02,15.77,39.60475,-9.085443,stormglass,2025-12-06 03:40:38.539180-08:00,11,3,3.690000,2025-12-07 03:00:00-08:00,2025-12-04 03:00:00-08:00
1,2025-12-04 04:00:00-08:00,3.63,3.41,5.39,15.78,39.60475,-9.085443,stormglass,2025-12-06 03:40:38.539180-08:00,12,3,3.660000,2025-12-07 03:00:00-08:00,2025-12-04 04:00:00-08:00
2,2025-12-04 05:00:00-08:00,3.61,3.42,5.18,15.79,39.60475,-9.085443,stormglass,2025-12-06 03:40:38.539180-08:00,13,3,3.643333,2025-12-07 03:00:00-08:00,2025-12-04 05:00:00-08:00
3,2025-12-04 06:00:00-08:00,3.58,3.43,4.97,15.79,39.60475,-9.085443,stormglass,2025-12-06 03:40:38.539180-08:00,14,3,3.606667,2025-12-07 03:00:00-08:00,2025-12-04 06:00:00-08:00
4,2025-12-04 07:00:00-08:00,3.55,3.44,4.76,15.79,39.60475,-9.085443,stormglass,2025-12-06 03:40:38.539180-08:00,15,3,3.580000,2025-12-07 03:00:00-08:00,2025-12-04 07:00:00-08:00


In [6]:
fig = px.line(
    pdf,
    x="timestamp",
    y="WAVE_HEIGHT",
    title="Wave Height Over Time (Last 7 Days)",
    labels={"timestamp": "Time (UTC)", "WAVE_HEIGHT": "Wave Height (m)"}
)
fig.show()


In [7]:
fig = px.histogram(
    pdf,
    x="WAVE_HEIGHT",
    nbins=30,
    title="Distribution of Wave Height",
    labels={"WAVE_HEIGHT": "Wave Height (m)"}
)
fig.show()


In [8]:
fig = px.scatter(
    pdf,
    x="WIND_SPEED",
    y="WAVE_HEIGHT",
    title="Wind Speed vs Wave Height",
    labels={"WIND_SPEED": "Wind Speed (m/s)", "WAVE_HEIGHT": "Wave Height (m)"}
)
fig.show()


In [9]:
daily_snow = (
    df_snow
    .with_column("DAY", F.to_date(F.col("TIMESTAMP")))
    .group_by("DAY")
    .agg(
        F.avg("WAVE_HEIGHT").alias("AVG_WAVE"),
        F.max("WAVE_HEIGHT").alias("MAX_WAVE"),
        F.avg("WIND_SPEED").alias("AVG_WIND")
    )
    .sort("DAY")
)

daily_pdf = daily_snow.to_pandas()
daily_pdf.head()


,DAY,AVG_WAVE,MAX_WAVE,AVG_WIND
0,2025-12-04,3.668095,4.21,3.722857
1,2025-12-05,4.025000,4.47,3.899583
2,2025-12-06,4.132083,4.69,3.330000
3,2025-12-07,3.180000,3.30,3.915000


In [10]:
fig = px.line(
    daily_pdf,
    x="DAY",
    y=["AVG_WAVE", "MAX_WAVE"],
    title="Daily Average and Max Wave Height",
    labels={"DAY": "Day", "value": "Wave Height (m)", "variable": "Metric"}
)
fig.show()


In [11]:
# Build hourly time-series from the 7-day window
ts = (
    pdf.set_index("timestamp")["WAVE_HEIGHT"]
    .sort_index()
    .resample("H")
    .mean()
    .interpolate()
)

ts.head(), ts.tail()


/var/folders/ky/9scsc6x10dlgg84f1fzcxdhc0000gn/T/ipykernel_94225/1382161468.py:5: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



(timestamp
 2025-12-04 03:00:00-08:00    3.69
 2025-12-04 04:00:00-08:00    3.63
 2025-12-04 05:00:00-08:00    3.61
 2025-12-04 06:00:00-08:00    3.58
 2025-12-04 07:00:00-08:00    3.55
 Freq: h, Name: WAVE_HEIGHT, dtype: float64,
 timestamp
 2025-12-06 23:00:00-08:00    3.38
 2025-12-07 00:00:00-08:00    3.30
 2025-12-07 01:00:00-08:00    3.22
 2025-12-07 02:00:00-08:00    3.14
 2025-12-07 03:00:00-08:00    3.06
 Freq: h, Name: WAVE_HEIGHT, dtype: float64)

## Notebook Summary

In this notebook we used **Snowpark for Python** to:

- Connect securely to Snowflake using environment variables  
- Load the `STORM_MARINE_CLEAN` table  
- Inspect schema, row counts, and date coverage  
- Compute descriptive statistics for wave, swell, wind, and temperature  
- Extract the last 7 days into a Pandas DataFrame  
- Visualise:
  - Wave height time series
  - Wave height distribution
  - Wind vs wave relationship
  - Daily average and maximum wave height  
- Build an hourly, gap-filled time series used later for the 24-hour forecasting model in the Streamlit dashboard.

This satisfies the **Notebook Analysis** requirement from the assignment (load table + explore dataset using Snowpark).
